In [ ]:
import mne
import matplotlib.pyplot as plt
import numpy as np
import MY_bcilib as mybci
from scipy.io import loadmat

被试者序号

In [ ]:
subject = 9

In [ ]:
'''
Train epoch 
——————————————————————————————————
1 : 400 
2 : 400
3 : 400
4 : 420 
5 : 420
6 : 400 
7 : 400
8 : 440
9 : 400

Test epoch 
——————————————————————————————————
1 : 320
2 : 280
3 : 320
4 : 320 
5 : 320
6 : 320 
7 : 320
8 : 320
9 : 320

'''
all_epoch_num = 400  #训练集共400个epoch
all_test_epoch_num = 320   # 共320个epoch

训练集数据与标签读取

In [ ]:
time_start = 2.5
time_end = 6
time_points = 876
train_data_r = np.zeros((all_epoch_num,3,time_points))  

epo_cnt = 0

for i in range(3):
    filename = 'data/B0' + str(subject) + '0'+ str( i + 1) +'T_epo.fif'
    epochs = mne.read_epochs(filename)

    epochs_crop = epochs.crop(time_start,time_end)

    data = epochs_crop.get_data()

    for i in range(data.shape[0]):
        train_data_r[epo_cnt,:,:] = data[i,:,:]
        epo_cnt += 1

In [ ]:
train_data_r.shape

In [ ]:
train_label_r = np.zeros((all_epoch_num,1))

cnt = 0
for i in range(3):
    filename = 'data/B0' + str(subject) + '0'+ str( i + 1) +'T.mat'
    raw_label_data = loadmat(filename)
    print( raw_label_data['classlabel'].shape)
    train_label_r[cnt:cnt+len(raw_label_data['classlabel']),:] = raw_label_data['classlabel'] - 1 
    cnt += len(raw_label_data['classlabel'])
    print(cnt)

In [ ]:
train_label_r = train_label_r.reshape(train_label_r.shape[0],)

train_label =  train_label_r

In [ ]:
print('train_label_r shape : ',train_label_r.shape)
print('train_data_r shape : ',train_data_r.shape)

测试集数据与标签读取

In [ ]:
test_data_r = np.zeros((all_test_epoch_num,3,time_points))  

epo_cnt = 0

for i in range(2):
    filename = 'data/B0' + str(subject) + '0'+ str( i + 4) +'E_epo.fif'
    epochs = mne.read_epochs(filename)

    epochs_crop = epochs.crop(time_start,time_end)

    data = epochs_crop.get_data()

    for i in range(data.shape[0]):
        test_data_r[epo_cnt,:,:] = data[i,:,:]
        epo_cnt += 1

In [ ]:
test_label_r = np.zeros((all_test_epoch_num,1))

cnt = 0
for i in range(2):
    filename = 'data/B0' + str(subject) + '0'+ str( i + 4) +'E.mat'
    raw_label_data = loadmat(filename)
    print( raw_label_data['classlabel'].shape)
    test_label_r[cnt:cnt+len(raw_label_data['classlabel']),:] = raw_label_data['classlabel'] - 1 
    cnt += len(raw_label_data['classlabel'])
    print(cnt)

In [ ]:
test_label_r = test_label_r.reshape(test_label_r.shape[0],)

test_label =  test_label_r

In [ ]:
print('test_label_r shape : ',test_label_r.shape)
print('test_data_r shape : ',test_data_r.shape)

数据预处理，带通滤波  1-4hz   4-8hz   8-13hz   13-30hz

In [ ]:
train_data_filt = np.zeros((all_epoch_num,4,3,876))

train_data_filt[:,0,:,:] = mybci.band_pass(train_data_r,1,4,sample_rate=250)
train_data_filt[:,1,:,:] = mybci.band_pass(train_data_r,4,8,sample_rate=250)
train_data_filt[:,2,:,:] = mybci.band_pass(train_data_r,8,13,sample_rate=250)
train_data_filt[:,3,:,:] = mybci.band_pass(train_data_r,13,30,sample_rate=250)

In [ ]:
test_data_filt = np.zeros((all_test_epoch_num,4,3,876))

test_data_filt[:,0,:,:] = mybci.band_pass(test_data_r,1,4,sample_rate=250)
test_data_filt[:,1,:,:] = mybci.band_pass(test_data_r,4,8,sample_rate=250)
test_data_filt[:,2,:,:] = mybci.band_pass(test_data_r,8,13,sample_rate=250)
test_data_filt[:,3,:,:] = mybci.band_pass(test_data_r,13,30,sample_rate=250)

CSP滤波

In [ ]:
csp_W = np.zeros((4,3,3))

for i in range(4):
    csp_W[i,:,:] = mybci.cal_W(train_data_filt[train_label==0,i,:,:],train_data_filt[train_label==1,i,:,:])



In [ ]:
train_data = np.zeros((all_epoch_num,4,3,876))

for i in range(4):
    train_data[:,i,:,:]  = mybci.apply_mix(csp_W[i,:,:],train_data_filt[:,i,:,:])

In [ ]:
test_data = np.zeros((all_test_epoch_num,4,3,876))

for i in range(4):
    test_data[:,i,:,:] = mybci.apply_mix(csp_W[i,:,:],test_data_filt[:,i,:,:])


In [ ]:
print('train_data shape : ,',train_data.shape)
print('test_data shape : ,',test_data.shape)

数据增强

In [ ]:
aug_times = 10


#-----------------------------------  left hand -----------------------------------------------------------
train_data_left_aug = np.zeros((int(all_epoch_num * (aug_times+1) / 2 ),4,3,876))
train_label_left_aug = np.zeros((int(all_epoch_num * (aug_times+1) / 2),4,))

print(train_data_left_aug.shape)

for i in range(4):
    train_data_left_aug[:,i,:,:],train_label_left_aug[:,i,]= mybci.data_augmentation(train_data[train_label==0,i,:,:],aug_times,15,handclass='left_hand')

#-----------------------------------  right hand -----------------------------------------------------------
train_data_right_aug = np.zeros((int(all_epoch_num * (aug_times+1) / 2),4,3,876))
train_label_right_aug = np.zeros((int(all_epoch_num * (aug_times+1) / 2),4,))

print(train_data_right_aug.shape)

for i in range(4):
    train_data_right_aug[:,i,:,:],train_label_right_aug[:,i,]= mybci.data_augmentation(train_data[train_label==1,i,:,:],aug_times,15,handclass='right_hand')


In [ ]:
train_data_aug = np.concatenate((train_data_left_aug,train_data_right_aug))

train_label_aug = np.concatenate((train_label_left_aug,train_label_right_aug))

train_label_aug = train_label_aug[:,0]


In [ ]:
train_data_aug = train_data_aug.reshape(train_data_aug.shape[0],train_data_aug.shape[1],1,train_data_aug.shape[2],train_data_aug.shape[3])

test_data = test_data.reshape(test_data.shape[0],test_data.shape[1],1,test_data.shape[2],test_data.shape[3])

In [ ]:
print('train_data_aug shape ',train_data_aug.shape)

print('train_label_aug shape ',train_label_aug.shape)

print('test_data shape ',test_data.shape)

print('test_label shape ',test_label.shape)

将标签one-hot编码，以便于后面交叉熵计算损失

In [ ]:
train_label_aug_onehot = np.zeros((train_label_aug.shape[0],2))

for i in range(train_label_aug.shape[0]):
    if(train_label_aug[i]==0):
        train_label_aug_onehot[i,0] = 1
    else:
        train_label_aug_onehot[i,1] = 1

In [ ]:
test_label_onehot = np.zeros((test_label.shape[0],2))

for i in range(test_label.shape[0]):
    if(test_label[i]==0):
        test_label_onehot[i,0] = 1
    else:
        test_label_onehot[i,1] = 1

In [ ]:
print('train_label_aug_onehot shape ',train_label_aug_onehot.shape)
print('test_label_onehot shape ',test_label_onehot.shape)

In [ ]:
train_label_aug_onehot[2200:2210,:]

创建dataloader

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset,TensorDataset

In [ ]:
batch_size = 128

#创建dataloader

data_train_tor = torch.from_numpy(train_data_aug)
data_train_tor = data_train_tor.float()
print(data_train_tor.size())


data_test_tor = torch.from_numpy(test_data)
data_test_tor = data_test_tor.float()
print(data_test_tor.size())


label_train_tor = torch.from_numpy(train_label_aug_onehot)
label_train_tor = label_train_tor.float()
label_test_tor = torch.from_numpy(test_label_onehot)
label_test_tor = label_test_tor.float()

#改成四维张量   样本数 * 通道数 * 高 * 宽 
#data_train_tor = data_train_tor.reshape(train_pic_data.shape[0],1,train_pic_data.shape[2],train_pic_data.shape[3])
print(data_train_tor.size())


#data_test_tor = data_test_tor.reshape(test_pic_data.shape[0],1,test_pic_data.shape[2],test_pic_data.shape[3])
print(data_test_tor.size())




train_dataset = TensorDataset(data_train_tor,label_train_tor)
test_dataset = TensorDataset(data_test_tor,label_test_tor)


train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

In [ ]:
for i,data in enumerate(train_loader):
    X = data[0]
    y = data[1]
    print(X.shape)


创建MS_CNN 模型

In [ ]:
class MSCNN_Net(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1,14,(1,5),(1,1),padding=(0,2)),  #in_channel , out_channel , kernel_size , stride
            nn.ReLU(),
            nn.MaxPool2d((1,15),(1,15)),
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(1,14,(1,3),(1,1),padding=(0,1)),  #in_channel , out_channel , kernel_size , stride
            nn.ReLU(),
            nn.MaxPool2d((1,15),(1,15)),
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(1,14,(1,1),(1,1)),  #in_channel , out_channel , kernel_size , stride
            nn.ReLU(),
            nn.MaxPool2d((1,15),(1,15)),
        )

        self.conv4 = nn.Sequential(
            nn.MaxPool2d((1,15),(1,15)),
            nn.Conv2d(1,14,(1,1),(1,1)),  #in_channel , out_channel , kernel_size , stride
            nn.ReLU(),
        )

        self.cl = nn.Sequential(
            nn.Conv2d(56,112,(1,1),(1,1)),  #in_channel , out_channel , kernel_size , stride
            nn.ReLU(), 
            nn.MaxPool2d((1,15),(1,15)),
        )

        self.fc = nn.Sequential(
            nn.Linear(448*3*3,128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128,32),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(32,2)

        )
    
    def forward(self,img):
        '''
        image : 5d-array (trials x freband x channels x height x width)
        '''
        

        feature1 = self.conv1(img[:,0,:,:,:])
        feature2 = self.conv2(img[:,0,:,:,:])
        feature3 = self.conv3(img[:,0,:,:,:])
        feature4 = self.conv4(img[:,0,:,:,:])
        
        feature = torch.cat((feature1,feature2,feature3,feature4),dim=1)

        fre1_feat = self.cl(feature)


        feature1 = self.conv1(img[:,1,:,:,:])
        feature2 = self.conv2(img[:,1,:,:,:])
        feature3 = self.conv3(img[:,1,:,:,:])
        feature4 = self.conv4(img[:,1,:,:,:])
        
        feature = torch.cat((feature1,feature2,feature3,feature4),dim=1)

        fre2_feat = self.cl(feature)

        feature1 = self.conv1(img[:,2,:,:,:])
        feature2 = self.conv2(img[:,2,:,:,:])
        feature3 = self.conv3(img[:,2,:,:,:])
        feature4 = self.conv4(img[:,2,:,:,:])
        
        feature = torch.cat((feature1,feature2,feature3,feature4),dim=1)

        fre3_feat = self.cl(feature)

        feature1 = self.conv1(img[:,3,:,:,:])
        feature2 = self.conv2(img[:,3,:,:,:])
        feature3 = self.conv3(img[:,3,:,:,:])
        feature4 = self.conv4(img[:,3,:,:,:])
        
        feature = torch.cat((feature1,feature2,feature3,feature4),dim=1)

        fre4_feat = self.cl(feature)

        feature_conb = torch.cat((fre1_feat,fre2_feat,fre3_feat,fre4_feat),dim=1)
        
        '''
        print('img size : ',img.shape)
        print('freband 1 feature shape: ',fre1_feat.size())
        print('feature shape: ',feature_conb.size())
        '''
        output = self.fc(feature_conb.reshape(img.shape[0],-1))
        return output

In [ ]:
net = MSCNN_Net()
X = torch.rand(2,4,1,3,876)
net(X)

In [ ]:
'''
net = MSCNN_Net()

X = torch.rand(1,1,3,876)
print(net)

for name,blk in net.named_children():
    X = blk(X)
    print(name,'  output shape :',X.shape)


'''

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
print(device)

In [ ]:
net = MSCNN_Net()

测试集预测结果

In [ ]:
import d2lzh_pytorch as d2l

lr,num_epochs = 0.0001,50
optimizer = torch.optim.Adam(net.parameters(),lr=lr)
d2l.train_ch5(net,train_loader,test_loader,batch_size,optimizer,device,num_epochs)
